In [1]:

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

from qiskit import *
from qiskit.circuit import Parameter
from qiskit import IBMQ, transpile
from qiskit.providers.ibmq.managed import IBMQJobManager

import numpy as np

import pickle

In [2]:
def scale_problems(problems, scale_min=-np.pi/4, scale_max=np.pi/4):
    scaled = []
    scaler = MinMaxScaler((-np.pi/4, np.pi/4))
    for entry in problems:
        scaled.append(np.array(scaler.fit_transform(entry.reshape(-1, 1)).flatten()))
    return scaled

In [3]:
def uncertainity_principle(circuit):
    circuit.h(range(circuit.width()))
    circuit.barrier()

def cost_encoding(circuit):
    for i in range(circuit.width()):
        circuit.ry(-Parameter('c'+str(i)), i)
    circuit.barrier()

def same_query_cost(circuit):
    for i in range(0,circuit.width(),2):
        circuit.crz(-np.pi/4,i,i+1)
    circuit.barrier()

def savings_encoding(circuit):
        for i in range(int(circuit.width()/2)):
            circuit.crz(Parameter('s'+str(i)+str(int(circuit.width()/2))), i, int(circuit.width()/2))
            circuit.crz(Parameter('s'+str(i)+str(int(1+circuit.width()/2))), i, int(1+circuit.width()/2))
        circuit.barrier()

def rx_layer(circuit, weight):
    circuit.rx(weight, range(circuit.width()))
    circuit.barrier()

def create_circuit(n_queries=2, n_plans=2, scheme="hcsx", xweight=np.pi/4):
    circuit = QuantumCircuit(n_queries*n_plans)
    for module in scheme:
        if module == "h":
            uncertainity_principle(circuit)
        elif module == "c":
            cost_encoding(circuit)
        elif module == "s":
            savings_encoding(circuit)
        elif module == "x":
            rx_layer(circuit, xweight)
    return circuit

In [4]:
def parse_results(result, max_value=True):
    remove_useless_keys(result)
    for i, key in enumerate(["0101", "1001", "0110", "1010"]):
        result[i] = result.pop(key, 0)
    if max_value:
        return max(result, key=result.get)
    else:
        return result

def remove_useless_keys(result):
    for key in ["0000","0001","0010","0100","1000","1101","1011","1100","0011","0111","1110","1111"]:
            if key in result:
                del result[key]
    for key in ["0101", "1001", "0110", "1010"]:
        if not key in result:
            result[key] = 0

def score_distance_results(results, solutions):
    x = np.array(results)
    y = np.array(solutions)
    distances = {}
    for r,s in zip(x,y):
        distance = np.where(s == r)[0]#[0]
        if len(distance) == 0:
            print('Solution not in prediction!')
            if not 'inf' in distances:
                distances['inf'] = 1
            else:
                distances['inf'] += 1
        elif not distance[0] in distances:
            distances[distance[0]] = 1
        else:
            distances[distance[0]] += 1
    return distances

In [5]:
def run_circuit(circuit, backend=Aer.get_backend("aer_simulator"), shots=1024):
    job = backend.run(transpile(circuit, backend), shots=shots)
    res = job.result()
    return res.get_counts(qc)

In [6]:
def score_results(results, solutions):
    return accuracy_score(results, solutions)

In [7]:
ibm_backend = 'ibmq_bogota'
total_iterations = 13

In [44]:
path = "data/1000_problems_with_solutions.p"
#path = "data/problems_with_solutions.p"
data = pickle.load(open(path, "rb"))

In [45]:
x_train = scale_problems(data['x_train'])
x_test = scale_problems(data['x_test'])
x = []
x.extend(x_train)
x.extend(x_test)
y_train = data['y_train']
y_train_labels = data['y_train_labels']
y_test = data['y_test']
y = []
y.extend(y_train_labels)
y.extend(y_test)

In [31]:
#load weights
weights_path = "experiments/optimizer_hyperparameters/amsgrad_final.p"
data = pickle.load(open(weights_path, "rb"))[0]

# ML OPTIMIZED SOLUTION

In [32]:
results = data['results'][0]
best_index = results['scoreTesting'].index(max(results['scoreTesting']))
rxweight = results['finalWeights'][best_index][0]
rxweight

0.23518365448276313

In [33]:
circuit = create_circuit(xweight=rxweight)

In [34]:
#prepare circuits for 1000 runs
circuits = []
for test in x:
    qc = circuit.copy().bind_parameters(test)
    qc.measure_all()
    circuits.append(qc)

In [13]:
#prepare circuits
circuits = []
for test in x_test:
    qc = circuit.copy().bind_parameters(test)
    qc.measure_all()
    circuits.append(qc)

### ML SIMULATION RUNS

In [35]:
data_to_save = {}

In [48]:
simulation_ml_accuracies = []
simulation_ml_parity_accuracies = []
simulation_ml_counts = []
for i in range(total_iterations):
    results_parity = []
    results_filtered = []
    c = []
    for qc in circuits:
        counts = run_circuit(qc.copy(), shots=4000)
        c.append(counts.copy())
        results_parity.append(int(max(counts, key=counts.get), 2) % 4)
        results_filtered.append(parse_results(counts))
    simulation_ml_accuracies.append(score_results(results_filtered, y))#y_test))
    simulation_ml_parity_accuracies.append(score_results(results_parity, y))#y_test))
    simulation_ml_counts.append(c)
data_to_save['ml_simulator_counts'] = simulation_ml_counts.copy() 

In [49]:
simulation_ml_filtered_counts = []
simulation_ml_parity_counts = []
for iteration in simulation_ml_counts:
    f = []
    p = []
    for counts in iteration:
        f.append(list({k: v for k, v in sorted(parse_results(counts.copy(), \
            max_value=False).items(), key=lambda item: item[1], reverse=True)}.keys()))
        parity_list = list({k: v for k, v in sorted(counts.items(), key=lambda item: item[1], reverse=True)}.keys())[:4]
        p.append([int(a, 2)%4 for a in parity_list])
    simulation_ml_filtered_counts.append(f)
    simulation_ml_parity_counts.append(p)

In [50]:
simulation_ml_filtered_distances = []
simulation_ml_parity_distances = []
distances = []
for f, p in zip(simulation_ml_filtered_counts, simulation_ml_parity_counts):
    simulation_ml_filtered_distances.append(score_distance_results(f, y))#y_test))
    simulation_ml_parity_distances.append(score_distance_results(p, y))#y_test))

Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in pred

### ML REAL HARDWARE RUNS

In [51]:
provider = IBMQ.load_account()
backend = provider.get_backend(ibm_backend)
job_manager = IBMQJobManager()

In [52]:
transpiled_circuits = []
for qc in circuits:
    transpiled_circuits.append(transpile(qc, backend=backend))

In [53]:
real_hardware_counts = []
real_hardware_accuracies_parity = []
real_hardware_accuracies = []
counts = []
for i in range(total_iterations):
    print('Iteration ' + str(i+1))
    job_set_static = job_manager.run(transpiled_circuits, backend=backend, name='Static MQO ML Solver Iteration ' + str(i))
    real_results = job_set_static.results()
    results_parity = []
    results_filtered = []
    c = []
    for j in range(len(transpiled_circuits)):
        counts = real_results.get_counts(j)
        c.append(counts.copy())
        results_parity.append(int(max(counts, key=counts.get), 2) % 4)
        results_filtered.append(parse_results(counts.copy()))
    real_hardware_accuracies.append(score_results(results_filtered, y))#y_test))
    real_hardware_accuracies_parity.append(score_results(results_parity, y))#y_test))
    real_hardware_counts.append(c)
data_to_save['ml_real_hw_counts'] = real_hardware_counts.copy() 

Iteration 1


managedjob._async_submit:WARNING:2022-05-23 22:12:53,473: Job limit reached, waiting for job 628beaba7fceac1730e58641 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-23 22:17:55,779: Job limit reached, waiting for job 628beaba7fceac1730e58641 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-23 22:22:57,772: Job limit reached, waiting for job 628beaba7fceac1730e58641 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-23 22:28:00,782: Job limit reached, waiting for job 628beaba7fceac1730e58641 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-23 22:32:20,562: Job limit reached, waiting for job 628beabd475b843dac18d228 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-23 22:34:40,238: Job limit reached, waiting for job 628beac07fceac6828e58643 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05

Iteration 2


managedjob._async_submit:WARNING:2022-05-23 22:55:22,007: Job limit reached, waiting for job 628bf4afe7e2a7dfda680faa to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-23 23:00:24,387: Job limit reached, waiting for job 628bf4afe7e2a7dfda680faa to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-23 23:02:50,786: Job limit reached, waiting for job 628bf4b17fe3b800701a148b to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-23 23:05:21,476: Job limit reached, waiting for job 628bf4b435cffcfff002da71 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-23 23:07:31,546: Job limit reached, waiting for job 628bf4b7e613289a86b04c4a to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-23 23:10:10,310: Job limit reached, waiting for job 628bf4b97fceac7ab0e58666 to finish before submitting the next one.


Iteration 3


...

managedjob._async_submit:WARNING:2022-05-24 05:03:01,371: Job limit reached, waiting for job 628c4ad87b770a3dcf138884 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 05:08:04,029: Job limit reached, waiting for job 628c4ad87b770a3dcf138884 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 05:09:57,841: Job limit reached, waiting for job 628c4adc475b84072618d34e to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 05:11:57,465: Job limit reached, waiting for job 628c4adfe61328ea83b04d23 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 05:13:57,539: Job limit reached, waiting for job 628c4ae27b770ac7a6138885 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 05:15:57,489: Job limit reached, waiting for job 628c4ae4d47c0282e191f9ba to finish before submitting the next one.


Iteration 12


managedjob._async_submit:WARNING:2022-05-24 05:36:20,758: Job limit reached, waiting for job 628c52a97b770a58411388a1 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 05:41:23,170: Job limit reached, waiting for job 628c52a97b770a58411388a1 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 05:46:25,500: Job limit reached, waiting for job 628c52a97b770a58411388a1 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 05:47:07,666: Job limit reached, waiting for job 628c52abeaf4c191ac62f493 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 05:49:07,550: Job limit reached, waiting for job 628c52aedf04f878363dbc16 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 05:51:08,926: Job limit reached, waiting for job 628c52b07b770a146d1388a2 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05

Iteration 13


managedjob._async_submit:WARNING:2022-05-24 06:28:25,337: Job limit reached, waiting for job 628c5ede475b84b71b18d383 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 06:33:27,771: Job limit reached, waiting for job 628c5ede475b84b71b18d383 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 06:38:30,337: Job limit reached, waiting for job 628c5ede475b84b71b18d383 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 06:43:32,970: Job limit reached, waiting for job 628c5ede475b84b71b18d383 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 06:48:35,402: Job limit reached, waiting for job 628c5ede475b84b71b18d383 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 06:53:37,537: Job limit reached, waiting for job 628c5ede475b84b71b18d383 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05

In [54]:
real_hw_ml_filtered_counts = []
real_hw_ml_parity_counts = []
for iteration in real_hardware_counts:
    f = []
    p = []
    for counts in iteration:
        f.append(list({k: v for k, v in sorted(parse_results(counts.copy(), \
            max_value=False).items(), key=lambda item: item[1], reverse=True)}.keys()))
        parity_list = list({k: v for k, v in sorted(counts.items(), key=lambda item: item[1], reverse=True)}.keys())[:4]
        p.append([int(a, 2)%4 for a in parity_list])
    real_hw_ml_filtered_counts.append(f)
    real_hw_ml_parity_counts.append(p)

In [55]:
real_hardware_ml_filtered_distances = []
real_hardware_ml_parity_distances = []
distances = []
for f, p in zip(real_hw_ml_filtered_counts, real_hw_ml_parity_counts):
    real_hardware_ml_filtered_distances.append(score_distance_results(f, y))#y_test))
    real_hardware_ml_parity_distances.append(score_distance_results(p, y))#y_test))

Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in prediction!
Solution not in pred

In [56]:
real_hardware_ml_parity_distances

[{'inf': 386, 0: 260, 1: 138, 2: 131, 3: 85},
 {'inf': 401, 0: 258, 1: 129, 3: 78, 2: 134},
 {'inf': 398, 0: 260, 1: 135, 3: 74, 2: 133},
 {'inf': 400, 0: 257, 1: 142, 3: 76, 2: 125},
 {'inf': 400, 0: 254, 1: 142, 2: 125, 3: 79},
 {'inf': 405, 0: 252, 1: 135, 2: 131, 3: 77},
 {'inf': 405, 0: 259, 1: 139, 3: 77, 2: 120},
 {'inf': 408, 0: 254, 1: 134, 3: 79, 2: 125},
 {'inf': 409, 0: 254, 1: 140, 2: 128, 3: 69},
 {'inf': 410, 0: 252, 1: 136, 3: 73, 2: 129},
 {'inf': 409, 0: 260, 1: 127, 3: 71, 2: 133},
 {'inf': 412, 0: 257, 1: 138, 2: 125, 3: 68},
 {'inf': 411, 0: 258, 1: 127, 3: 81, 2: 123}]

# STATIC CIRCUIT RUNS

In [57]:
circuit = create_circuit()
#prepare circuits
circuits = []
for test in x:#x_test:
    qc = circuit.copy().bind_parameters(test)
    qc.measure_all()
    circuits.append(qc)

transpiled_circuits = []
for qc in circuits:
    transpiled_circuits.append(transpile(qc, backend=backend))

### SIMULATOR STATIC RUNS

In [58]:
simulation_accuracies = []
simulation_counts = []

for i in range(total_iterations):
    results = []
    distance_results = []
    c = []
    for qc in circuits:
        counts = run_circuit(qc.copy(), shots=4000)
        c.append(counts)
        results.append(parse_results(counts.copy()))
    simulation_accuracies.append(score_results(results, y))#y_test))
    simulation_counts.append(c)
data_to_save['static_simulation_counts'] = simulation_counts.copy() 

In [59]:
simulation_distances = []
for f in simulation_counts:
        res = []
        for r in f:
                x = parse_results(r.copy(), max_value=False)
                x = list({k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}.keys())
                res.append(x)
        simulation_distances.append(score_distance_results(res, y))#y_test))

In [60]:
simulation_distances

[{0: 858, 1: 123, 2: 17, 3: 2},
 {0: 855, 1: 122, 2: 21, 3: 2},
 {0: 859, 2: 21, 1: 118, 3: 2},
 {0: 864, 1: 120, 2: 15, 3: 1},
 {0: 869, 1: 111, 3: 2, 2: 18},
 {0: 859, 1: 119, 3: 3, 2: 19},
 {0: 853, 1: 125, 2: 21, 3: 1},
 {0: 854, 1: 127, 2: 18, 3: 1},
 {0: 862, 1: 117, 2: 18, 3: 3},
 {0: 847, 1: 132, 3: 2, 2: 19},
 {0: 859, 1: 116, 3: 2, 2: 23},
 {0: 857, 2: 18, 1: 123, 3: 2},
 {0: 854, 1: 127, 3: 2, 2: 17}]

### REAL HARDWARE STATIC RUNS

In [61]:
real_accuracies = []
real_counts = []
for i in range(total_iterations):
    print('Iteration ' + str(i+1))
    job_set_static = job_manager.run(transpiled_circuits, backend=backend, name='Static MQO Solver Iteration ' + str(i))
    real_results = job_set_static.results()
    res_temp = []
    for j in range(len(transpiled_circuits)):
        real_counts.append(real_results.get_counts(j).copy())
        res_temp.append(parse_results(real_results.get_counts(j)))
    real_accuracies.append(score_results(res_temp, y))#y_test))
data_to_save['static_real_hw_counts'] = real_counts.copy() 


Iteration 1


managedjob._async_submit:WARNING:2022-05-24 08:30:48,959: Job limit reached, waiting for job 628c7b8f7fe3b862e41a15f9 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 08:35:51,690: Job limit reached, waiting for job 628c7b8f7fe3b862e41a15f9 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 08:37:08,992: Job limit reached, waiting for job 628c7b917fceac912ee587d3 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 08:39:08,889: Job limit reached, waiting for job 628c7b947b770a55351388fe to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 08:41:08,868: Job limit reached, waiting for job 628c7b967b770af2b31388ff to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 08:43:11,374: Job limit reached, waiting for job 628c7b997fceac8028e587d5 to finish before submitting the next one.


Iteration 2


managedjob._async_submit:WARNING:2022-05-24 09:04:31,195: Job limit reached, waiting for job 628c8374475b846aab18d3f1 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 09:09:33,462: Job limit reached, waiting for job 628c8374475b846aab18d3f1 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 09:14:36,218: Job limit reached, waiting for job 628c8374475b846aab18d3f1 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 09:16:59,369: Job limit reached, waiting for job 628c83777fceac170be58806 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 09:19:09,458: Job limit reached, waiting for job 628c837ae7e2a7593968113f to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 09:21:08,995: Job limit reached, waiting for job 628c837ceaf4c11ead62f539 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05

Iteration 3


...

managedjob._async_submit:WARNING:2022-05-24 15:15:33,529: Job limit reached, waiting for job 628cda6944b4037db9c60333 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 15:20:37,050: Job limit reached, waiting for job 628cda6944b4037db9c60333 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 15:25:23,008: Job limit reached, waiting for job 628cda6b891f1a1fd64b9063 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 15:27:18,465: Job limit reached, waiting for job 628cda6e1d9903fd92e49301 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 15:29:18,265: Job limit reached, waiting for job 628cda71c8f7494e0dcefbbb to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 15:31:28,396: Job limit reached, waiting for job 628cda74ed5c01e5e3dfd1bd to finish before submitting the next one.


Iteration 12


managedjob._async_submit:WARNING:2022-05-24 15:46:44,360: Job limit reached, waiting for job 628ce1b5ed5c014e8edfd1e1 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 15:51:47,557: Job limit reached, waiting for job 628ce1b5ed5c014e8edfd1e1 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 15:56:50,741: Job limit reached, waiting for job 628ce1b5ed5c014e8edfd1e1 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 16:01:27,775: Job limit reached, waiting for job 628ce1b8891f1a395a4b9080 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 16:03:28,528: Job limit reached, waiting for job 628ce1bc8f760335d26d190b to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 16:05:27,762: Job limit reached, waiting for job 628ce1c0ed5c018a71dfd1e2 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05

Iteration 13


managedjob._async_submit:WARNING:2022-05-24 16:22:04,524: Job limit reached, waiting for job 628ce9ff891f1a1f524b90a6 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 16:27:07,963: Job limit reached, waiting for job 628ce9ff891f1a1f524b90a6 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 16:27:27,860: Job limit reached, waiting for job 628cea033bb6a9951f7c1f0b to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 16:29:38,042: Job limit reached, waiting for job 628cea05f148606278643cdf to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 16:31:38,480: Job limit reached, waiting for job 628cea09891f1a79204b90a8 to finish before submitting the next one.
managedjob._async_submit:WARNING:2022-05-24 16:33:37,610: Job limit reached, waiting for job 628cea0c909d22d2c83d5fb5 to finish before submitting the next one.


In [63]:
real_hardware_distances = []
for f in real_counts:
        res = []
        for r in f:
                x = parse_results(r.copy(), max_value=False)
                x = list({k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}.keys())
                res.append(x)
        real_hardware_distances.append(score_distance_results(res, y))#y_test))

AttributeError: 'str' object has no attribute 'copy'

In [64]:
real_hardware_distances

[]

In [65]:
path = 'experiments/final_runs/'
savename = '1000_runs_done_accuracies.p'
try:
    os.makedirs(path)
except:
    print('path already exists!')

path already exists!


In [66]:
run_data = {'static_simulator_filtered_accuracy':simulation_accuracies,
            'static_simulator_filtered_distances':simulation_distances, 
            'static_real_hw_filtered_accuracy': real_accuracies, 
            'static_real_hw_filtered_distances':real_hardware_distances,
            'ml_simulator_filtered_accuracy': simulation_ml_accuracies, 
            'ml_simulator_parity_accuracy': simulation_ml_parity_accuracies,
            'ml_simulator_filtered_distances': simulation_ml_filtered_distances, 
            'ml_simulator_parity_distances': simulation_ml_parity_distances,
            'ml_real_hw_filtered_accuracy': real_hardware_accuracies, 
            'ml_real_hw_parity_accuracy': real_hardware_accuracies_parity,
            'ml_real_hw_filtered_distances': real_hardware_ml_filtered_distances, 
            'ml_real_hw_parity_distances':real_hardware_ml_parity_distances,}

In [68]:
run_data

{'static_simulator_filtered_accuracy': [0.858,
  0.855,
  0.859,
  0.864,
  0.869,
  0.859,
  0.853,
  0.854,
  0.862,
  0.847,
  0.859,
  0.857,
  0.854],
 'static_simulator_filtered_distances': [{0: 858, 1: 123, 2: 17, 3: 2},
  {0: 855, 1: 122, 2: 21, 3: 2},
  {0: 859, 2: 21, 1: 118, 3: 2},
  {0: 864, 1: 120, 2: 15, 3: 1},
  {0: 869, 1: 111, 3: 2, 2: 18},
  {0: 859, 1: 119, 3: 3, 2: 19},
  {0: 853, 1: 125, 2: 21, 3: 1},
  {0: 854, 1: 127, 2: 18, 3: 1},
  {0: 862, 1: 117, 2: 18, 3: 3},
  {0: 847, 1: 132, 3: 2, 2: 19},
  {0: 859, 1: 116, 3: 2, 2: 23},
  {0: 857, 2: 18, 1: 123, 3: 2},
  {0: 854, 1: 127, 3: 2, 2: 17}],
 'static_real_hw_filtered_accuracy': [0.76,
  0.744,
  0.73,
  0.724,
  0.738,
  0.724,
  0.724,
  0.729,
  0.736,
  0.74,
  0.722,
  0.734,
  0.726],
 'static_real_hw_filtered_distances': [],
 'ml_simulator_filtered_accuracy': [0.237,
  0.233,
  0.237,
  0.235,
  0.24,
  0.239,
  0.235,
  0.239,
  0.239,
  0.243,
  0.233,
  0.237,
  0.239],
 'ml_simulator_parity_accuracy'

In [67]:

pickle.dump(run_data, open( path+savename, "wb" ) )    

In [31]:
data_to_save.keys()

dict_keys(['ml_simulator_counts', 'ml_real_hw_counts', 'static_simulation_counts', 'static_real_hw_counts'])

In [ ]:

pickle.dump(data_to_save, open( path+"counts_of_1000_runs.p", "wb" ) )    